In [334]:
import redis
import numpy as np

In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

# Your API Key setup
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("Google API Key is missing. Please set it in the .env file.")

# Configure the Generative AI library with the API key
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("api key configured")


api key configured


c:\Users\snuin\anaconda3\envs\env-translate\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
redis_client = redis.Redis(host='localhost', port=6379, db=0)

In [336]:
redis_client

<redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(db=0,username=None,password=None,socket_timeout=None,encoding=utf-8,encoding_errors=strict,decode_responses=False,retry_on_error=[],retry=<redis.retry.Retry object at 0x000001B91547AEB0>,health_check_interval=0,client_name=None,lib_name=redis-py,lib_version=6.4.0,redis_connect_func=None,credential_provider=None,protocol=2,host=localhost,port=6379,socket_connect_timeout=None,socket_keepalive=None,socket_keepalive_options=None)>)>)>

In [337]:
## verify my python redis_client can talk to redis server inside Docker Container
print(redis_client.ping())  

True


In [338]:
import PyPDF2
from sentence_transformers import SentenceTransformer

In [339]:
#pdffile_1 = open("C:\\Work Drive-Suchi\\Mithileshsir_AI_ML_GenAI\\Project\\by me\\ChatBot_v2\\AMFI_MF",'rb')
pdffile_1 = open("AMFI_MF.pdf",'rb')
pdf_reader = PyPDF2.PdfReader(pdffile_1)
pdf_reader

In [340]:
len(pdf_reader.pages)

17

In [341]:
pdf_text = ""

for page in pdf_reader.pages:
    pdf_text += page.extract_text()

pdf_text

print(len(pdf_text))

46721


In [342]:
pdf_text

' \n \n \n \n \n \nFAQs for Mutual Fund Investors  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nDISCLAIMER • This is not a legal document. • These FAQs are  prepared with a v iew to guide mutual \nfund Investors on investments in mutual funds . For full particulars of  laws governing the Mutual Funds \n(MF), please refer to the  Acts/Regulations/Guidelines/Circulars etc. appearing under the Legal tab of \nSEBI website i.e. www.sebi.g ov.in.  The contents of these FAQs are updated as on August 31, 2024 . \n \n \n \n \n \n \n \n \n \n Introduction  \n \nDifferent investment avenues are available to investors. Mutual funds also offer good investment \nopportunities to the investors. Like all investments, they also carry certain risks. The investors \nshould compare the risks and expected returns after adjustment of tax on various instruments \nwhile taking investment decisions. The investors may seek advice from experts while making \ninvestm

In [343]:
import re

In [344]:
## Not sure about preprocessing :

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)   # collapse all whitespace
    return text.strip()

cleaned_text = clean_text(pdf_text)
len(cleaned_text)

45080

In [ ]:
# words = cleaned_text.split()
# print(len(words))
# print(words)

7617
['FAQs', 'for', 'Mutual', 'Fund', 'Investors', 'DISCLAIMER', '•', 'This', 'is', 'not', 'a', 'legal', 'document.', '•', 'These', 'FAQs', 'are', 'prepared', 'with', 'a', 'v', 'iew', 'to', 'guide', 'mutual', 'fund', 'Investors', 'on', 'investments', 'in', 'mutual', 'funds', '.', 'For', 'full', 'particulars', 'of', 'laws', 'governing', 'the', 'Mutual', 'Funds', '(MF),', 'please', 'refer', 'to', 'the', 'Acts/Regulations/Guidelines/Circulars', 'etc.', 'appearing', 'under', 'the', 'Legal', 'tab', 'of', 'SEBI', 'website', 'i.e.', 'www.sebi.g', 'ov.in.', 'The', 'contents', 'of', 'these', 'FAQs', 'are', 'updated', 'as', 'on', 'August', '31,', '2024', '.', 'Introduction', 'Different', 'investment', 'avenues', 'are', 'available', 'to', 'investors.', 'Mutual', 'funds', 'also', 'offer', 'good', 'investment', 'opportunities', 'to', 'the', 'investors.', 'Like', 'all', 'investments,', 'they', 'also', 'carry', 'certain', 'risks.', 'The', 'investors', 'should', 'compare', 'the', 'risks', 'and', 'exp

In [346]:
## Langchain is big package so with specification I installed

#!pip install langchain-text-splitters

In [352]:
import langchain_text_splitters

In [354]:
## character-based chunks with overlap:

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_text(cleaned_text)


print(f"Total chunks: {len(chunks)}")
print(chunks[0])  # preview first chunk

Total chunks: 181
FAQs for Mutual Fund Investors DISCLAIMER • This is not a legal document. • These FAQs are prepared with a v iew to guide mutual fund Investors on investments in mutual funds . For full particulars of laws governing the Mutual Funds (MF), please refer to the Acts/Regulations/Guidelines/Circulars


In [314]:
# Function to split into chunks
def chunk_text(text, chunk_size=100, min_words=10):
    """
    text: cleaned text
    chunk_size: number of words per chunk
    min_words: skip chunks with fewer words
    """
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        print("chunk :",chunk)
        if len(chunk.split()) >= min_words:  # skip tiny chunks
            chunks.append(chunk)
    return chunks

In [355]:
# # Chunk text
# chunks = chunk_text(cleaned_text, chunk_size=100, min_words=10)
# print(f"Total meaningful chunks: {len(chunks)}")

# # Preview first chunk
# print(chunks[0])

In [190]:
len(words)

7617

Entire cleaned PDF text (all 17 pages) has been split into 77 meaningful chunks based on your chunk_size=100 and min_words=10 settings.
Each chunk is roughly 100 words (except possibly the last chunk).

Only chunks with at least 10 words were kept (tiny headings or blank lines were skipped).

These 77 chunks are what you’ll create embeddings for and store in Redis.


So instead of embedding 17 pages as a whole (which would be too coarse), you now have 77 smaller, contextually meaningful pieces — perfect for semantic search.

In [ ]:
# # Read PDF
# reader = PyPDF2.PdfReader("sample.pdf")
# raw_text = " ".join([page.extract_text() for page in reader.pages])

# # Clean text
# cleaned_text = clean_text(raw_text)
# print(f"Cleaned text length: {len(cleaned_text)} chars")

# # Chunk text
# chunks = chunk_text(cleaned_text, chunk_size=100, min_words=10)
# print(f"Total meaningful chunks: {len(chunks)}")

# # Preview first chunk
# print(chunks[0])

In [356]:
## Create embeddings

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embedding = model.encode(chunks, convert_to_numpy=True).astype('float32')  # FAISS requires float32

doc_embedding

array([[ 0.0674909 , -0.06122107, -0.07691956, ..., -0.01994646,
         0.05195857,  0.02238643],
       [ 0.07119033, -0.00604787, -0.07137781, ..., -0.03220365,
         0.03060421, -0.01128874],
       [ 0.05933017, -0.00652998, -0.00220402, ..., -0.03095852,
         0.01604361,  0.01922919],
       ...,
       [ 0.02176405,  0.07803318, -0.01382113, ..., -0.06328876,
        -0.02423086, -0.00933218],
       [-0.00164257,  0.05715283, -0.0654509 , ..., -0.0666481 ,
        -0.07199147,  0.02732177],
       [-0.03306865,  0.02556622, -0.09139819, ..., -0.05908464,
        -0.0623414 ,  0.02269478]], dtype=float32)

In [357]:
doc_embedding.shape

(181, 384)

In [358]:
# Build FAISS index
import faiss

dim = doc_embedding.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(doc_embedding)

In [359]:
## use mithilesh sir approach....RAG

#doc_texts = chunks
doc_texts = cleaned_text

def retrieve_docs(query, top_k=3):
    print("From fucntion retrive_docs :", query)
    query_emb = model.encode([query])
    D, I = index.search(np.array(query_emb), top_k)
    print("Distances:", D)
    print("Indexes:", I[0])
 
    retrieved = [doc_texts[i] for i in I[0]]
    print("retrived from function --> : ", retrieved)
    return retrieved


In [361]:
def generate_answer(query, context):
    #return f"Based on context: {context[0]} \nAnswer: {query} → {context[0]}"
    combined_context = " ".join(context)
    return f"Based on context: {combined_context}\nAnswer: {query} → {combined_context}"

In [363]:
## redis get and set

def rag_with_cache(query):
    # Step 1: Check Redis cache
    print("Query from function: ", query)
    cached_answer = redis_client.get(query)
    print("Cached_Answer from function :", cached_answer)
    if cached_answer:
        print("[Answer from Redis] :")
        return f"[CACHE HIT] {cached_answer.decode()}"

    # Step 2: If not cached → do RAG
    context = retrieve_docs(query)
    answer = generate_answer(query, context)
    print("Answer from RAG - Knowledge base : ",answer )

    # Step 3: Store in Redis cache
    #redis_client.set(query, answer)  # expires in 60s
    redis_client.set(query, answer, ex=60)  # expires in 60s
    return f"[RAG GENERATED] {answer}"

🔹 Typical setup for a chatbot

FAISS → semantic search for PDF / knowledge base chunks

Redis cache → speed up repeated queries and optionally store previous conversation context

Optional: Redis full storage for embeddings if you want persistence + multi-user memory

✅ Conclusion:

For a single-user or small chatbot, Redis as cache + FAISS for embeddings is enough.

You don’t need to store embeddings in Redis unless you need persistence or multi-user access.

In [365]:
queries = ["Suggest different types of funds available " ,
           "what is Balanced or hybrid funds",
           "who is Authorities to handle mf" ,
           "how to start investing in MF industries",
           "what is Balanced or hybrid funds",
            "what is FoF Scheme" ]

print(" *** Demo *** RAG+CAG with or without Latency \n ")


for q in queries:
    print(f"\nQ: {q}")  
    answer = rag_with_cache(q)
    print(answer)
    print()

 *** Demo *** RAG+CAG with or without Latency 
 

Q: Suggest different types of funds available 
Query from function:  Suggest different types of funds available 
Cached_Answer from function : b'Based on context: Pharmaceuticals etc. These funds are launched to get the benefit from growth and performance of the particular sector . For example, technology sector would primarily invest in tech companies only. The returns in these funds are dependent on the performance of t he respective sectors/industry. Thematic funds invest in stocks across a specific theme. These funds focus on specific themes or trends in the market. The themes can be based on economic trends, social changes, technological advancements etc. Examples of these funds are MNC funds, PSU (public sector units) funds, energy funds . Compare d to sectoral fund thematic funds \nAnswer: Suggest different types of funds available  \xe2\x86\x92 Pharmaceuticals etc. These funds are launched to get the benefit from growth and perf

In [368]:
query ="what about exit loads"
# print(query)
answer = rag_with_cache(query)
print(answer)



Query from function:  what about exit loads
Cached_Answer from function : None
From fucntion retrive_docs : what about exit loads
Distances: [[0.8191251 0.8858646 0.9649209]]
Indexes: [75 74 76]
retrived from function --> :  [' ', '•', 'T']
Answer from RAG - Knowledge base :  Based on context:   • T
Answer: what about exit loads →   • T
[RAG GENERATED] Based on context:   • T
Answer: what about exit loads →   • T


In [309]:
que = "what is Liquidity?"
print(que)
print(" *** Demo *** RAG+CAG with or without Latency ")
for q1 in que:
    print(q1)
    ans = rag_with_cache(q1)
    print(ans)

what is Liquidity?
 *** Demo *** RAG+CAG with or without Latency 
w
Query from function:  w
Cached_Answer from function : None
From fucntion retrive_docs : w
Distannce and Indexes: [0]
FAQs for Mutual Fund Investors DISCLAIMER • This is not a legal document. • These FAQs are prepared with a v iew to guide mutual fund Investors on investments in mutual funds . For full particulars of laws governing the Mutual Funds (MF), please refer to the Acts/Regulations/Guidelines/Circulars etc. appearing under the Legal tab of SEBI website i.e. www.sebi.g ov.in. The contents of these FAQs are updated as on August 31, 2024 . Introduction Different investment avenues are available to investors. Mutual funds also offer good investment opportunities to the investors. Like all investments, they also carry certain risks. The investors should compare the risks and expected returns after adjustment of tax on various instruments while taking investment decisions. The investors may seek advice from experts w

In [219]:
## Q. Check with FoF schems:

### cleaned_text->chunks --> (77) Embedding -- not proper output -- check with FoF schemes
### cleaned_text->words -->(7677) embedding -- not proper output -- got output but with previous attached chunk too much and not get the exact answer of FOF only one sent and second sent cut off
